In [50]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrame
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
import time

# Create spark session
spark = SparkSession.builder.appName("Intrusion_Detector").config("spark.some.config.option", "some-value").getOrCreate()
dataFrameReader = spark.read

filename = "in/Monday-WorkingHours.pcap_ISCX.csv"
filePath = "in/test.csv"
# session = SparkContext(appName="Intrusion_Detector")
# dataFrameReader = session.read

In [56]:
selected_features = [' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 
                     ' Fwd Packet Length Mean', ' Fwd Packet Length Std', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 
                     ' Flow Bytes/s', ' Flow IAT Mean', ' Flow IAT Std', ' Bwd IAT Mean', ' Bwd IAT Std',' Label']

df = spark.read.option("header","true").option("inferSchema", value=True).csv(filename)
data = df.select(selected_features).dropna().rdd

features_labels = data.map(lambda row:  ( array([float(x) for x in row[:-1] ]), row[-1] ))
train_data = features_labels.map(lambda tup: tup[0]).cache()
labels = features_labels.map(lambda tup: tup[1]).cache()
print(train_data.take(1))
print(labels.take(1))

t1 = time.time()
clusters = KMeans.train(train_data, 2)
print(time.time() - t1)

[array([ 4.9188e+04,  4.0000e+00,  2.0000e+00,  0.0000e+00,  1.2000e+01,
        0.0000e+00,  6.0000e+00,  6.0000e+00,  6.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  3.0000e+06,
        5.0000e+05,  4.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,
        4.0000e+00,  4.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.0000e+01,
        0.0000e+00,  5.0000e+05,  0.0000e+00,  6.0000e+00,  6.0000e+00,
        6.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  9.0000e+00,  6.0000e+00,  0.0000e+00,
        4.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  2.0000e+00,  1.2000e+01,  0.0000e+00,
        0.0000e+00,  3.2900e+02, -1.0000e+00,  1.0000e+00,  2.0

In [57]:
data = spark.read.option("header","true").option("inferSchema", value=True).csv(filename).dropna().rdd

features_labels = data.map(lambda row:  ( array([float(x) for x in row[:-1] ]), row[-1] ))
train_data = features_labels.map(lambda tup: tup[0]).cache()
labels = features_labels.map(lambda tup: tup[1]).cache()
print(train_data.take(1))
print(labels.take(1))

t1 = time.time()
clusters = KMeans.train(train_data, 2)
print(time.time() - t1)

[array([ 4.9188e+04,  4.0000e+00,  2.0000e+00,  0.0000e+00,  1.2000e+01,
        0.0000e+00,  6.0000e+00,  6.0000e+00,  6.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  3.0000e+06,
        5.0000e+05,  4.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,
        4.0000e+00,  4.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.0000e+01,
        0.0000e+00,  5.0000e+05,  0.0000e+00,  6.0000e+00,  6.0000e+00,
        6.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  9.0000e+00,  6.0000e+00,  0.0000e+00,
        4.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  2.0000e+00,  1.2000e+01,  0.0000e+00,
        0.0000e+00,  3.2900e+02, -1.0000e+00,  1.0000e+00,  2.0

In [54]:
# [1] Preprocessing

# load data
df = dataFrameReader.option("header", "true").option("inferSchema", value = True).csv(filePath)

# # Inspect data
# print("=== Print out schema ===")
# df.printSchema()
# (n, m) = (df.count(), len(df.columns))
# print(n, m)

# Destination Port, Flow Duration, Total Fwd Packets, Total Backward Packets, Fwd Packet Length Mean, 
# Fwd Packet Length Std, Bwd Packet Length Mean, Bwd Packet Length Std, Flow Bytes/s, Flow IAT Mean, 
# Flow IAT Std, Bwd IAT Mean, Bwd IAT Std, Fwd PSH Flags, Bwd PSH Flags, Fwd URG Flags, Bwd URG Flags, 
# Fwd Header Length, Bwd Header Length

# split data frame rows into features vectors and labels
lbls = df[-1]
assembler = VectorAssembler( inputCols= df.schema.names[:-1], outputCol='features' )
df_transformed= assembler.transform(df)
data = df_transformed.select('features')
# lbls.take(5)

t1 = time.time()
clusters = KMeans.train(data.rdd, k=2)
print(time.time() - t1)

Py4JJavaError: An error occurred while calling o2926.trainKMeansModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 190.0 failed 1 times, most recent failure: Lost task 0.0 in stage 190.0 (TID 465, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 253, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 248, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\python\lib\pyspark.zip\pyspark\util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "C:\spark\python\lib\pyspark.zip\pyspark\mllib\linalg\__init__.py", line 83, in _convert_to_vector
    raise TypeError("Cannot convert type %s into Vector" % type(l))
TypeError: Cannot convert type <class 'pyspark.sql.types.Row'> into Vecto

In [112]:
# load data
df = dataFrameReader.option("header", "true").option("inferSchema", value = True)\
.csv("in/Monday-WorkingHours.pcap_ISCX.csv").rdd
parsedData = df.map(lambda line: ( array([float(x) for x in line[:-1]]), line[-1] ) )
data = parsedData.map(lambda row: row[0]).cache()
lbls = parsedData.map(lambda row: row[1]).cache()
print(data.take(1))
# print(lbls.take(2))

t1 = time.time()
clusters = KMeans.train(data, 2, maxIterations=10, runs=2, initializationMode="random")
print(time.time() - t1)


[array([ 4.9188e+04,  4.0000e+00,  2.0000e+00,  0.0000e+00,  1.2000e+01,
        0.0000e+00,  6.0000e+00,  6.0000e+00,  6.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  3.0000e+06,
        5.0000e+05,  4.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,
        4.0000e+00,  4.0000e+00,  0.0000e+00,  4.0000e+00,  4.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.0000e+01,
        0.0000e+00,  5.0000e+05,  0.0000e+00,  6.0000e+00,  6.0000e+00,
        6.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  9.0000e+00,  6.0000e+00,  0.0000e+00,
        4.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  2.0000e+00,  1.2000e+01,  0.0000e+00,
        0.0000e+00,  3.2900e+02, -1.0000e+00,  1.0000e+00,  2.0

IllegalArgumentException: 'requirement failed'

In [38]:
sc = SparkContext(appName="KMeansClustering")  # SparkContext
raw_data = spark.textFile("in/Monday-WorkingHours.pcap_ISCX.csv", 6)

def parseLine(line):
    cols = line.split(',')
    # label is the last column
    label = cols[-1]
    
    # vector is every column, except the label
    vector = cols[:-1]
    
    # delete values of columns that have index 1->3 (categorical features)
    vector = [element for i, element in enumerate(vector) if i not in [1, 2, 3]]
    
    # convert each value from string to float
    vector = np.array(vector, dtype=np.float)
    
    return (label, vector)

labelsAndData = raw_data.map(parseLine).cache()

# we only need the data, not the label
data = labelsAndData.map(lambda row: row[0]).cache()

# number of connections
n = data.count()

t1 = time.time()
clusters = KMeans.train(data, 2, maxIterations=10, runs=10, initializationMode="random")
print(time.time() - t1)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Intrusion_Detector, master=local[*]) created by __init__ at <ipython-input-1-7021e3757e5f>:13 